In [266]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [267]:
df = pd.read_excel('shopee_data.xlsx', index_col=0)

In [268]:
columns_tolist = df.columns.tolist()
columns_tolist

['Status Pesanan',
 'Alasan Pembatalan',
 'Status Pembatalan/ Pengembalian',
 'No. Resi',
 'Opsi Pengiriman',
 'Antar ke counter/ pick-up',
 'Pesanan Harus Dikirimkan Sebelum (Menghindari keterlambatan)',
 'Waktu Pengiriman Diatur',
 'Waktu Pesanan Dibuat',
 'Waktu Pembayaran Dilakukan',
 'Metode Pembayaran',
 'SKU Induk',
 'Nama Produk',
 'Nomor Referensi SKU',
 'Nama Variasi',
 'Harga Awal',
 'Harga Setelah Diskon',
 'Jumlah',
 'Returned quantity',
 'Total Harga Produk',
 'Total Diskon',
 'Diskon Dari Penjual',
 'Diskon Dari Shopee',
 'Berat Produk',
 'Jumlah Produk di Pesan',
 'Total Berat',
 'Voucher Ditanggung Penjual',
 'Cashback Koin',
 'Voucher Ditanggung Shopee',
 'Paket Diskon',
 'Paket Diskon (Diskon dari Shopee)',
 'Paket Diskon (Diskon dari Penjual)',
 'Potongan Koin Shopee',
 'Diskon Kartu Kredit',
 'Ongkos Kirim Dibayar oleh Pembeli',
 'Estimasi Potongan Biaya Pengiriman',
 'Ongkos Kirim Pengembalian Barang',
 'Total Pembayaran',
 'Perkiraan Ongkos Kirim',
 'Catatan dari

In [269]:
# Drop kolom yang tidak diperlukan langsung pada df tanpa variabel baru
df.drop(columns=[
    # 'Status Pesanan',
    'Alasan Pembatalan',
    'Status Pembatalan/ Pengembalian',
    'No. Resi',
    'Opsi Pengiriman',
    'Antar ke counter/ pick-up',
    'Pesanan Harus Dikirimkan Sebelum (Menghindari keterlambatan)',
    'Waktu Pengiriman Diatur',
    'Waktu Pembayaran Dilakukan',
    'Metode Pembayaran',
    'SKU Induk',
    'Nomor Referensi SKU',
    'Nama Variasi',
    'Returned quantity',
    'Total Diskon',
    'Berat Produk',
    'Jumlah Produk di Pesan',
    'Total Berat',
    'Cashback Koin',
    'Paket Diskon',
    'Paket Diskon (Diskon dari Shopee)',
    'Paket Diskon (Diskon dari Penjual)',
    'Potongan Koin Shopee',
    'Diskon Kartu Kredit',
    'Ongkos Kirim Dibayar oleh Pembeli',
    'Estimasi Potongan Biaya Pengiriman',
    'Ongkos Kirim Pengembalian Barang',
    'Total Pembayaran',
    'Perkiraan Ongkos Kirim',
    'Catatan dari Pembeli',
    'Catatan',
    'Nama Penerima',
    'No. Telepon',
    'Alamat Pengiriman',
    'Waktu Pesanan Selesai'
], inplace=True)


In [270]:
df = df[df['Status Pesanan'] != 'Batal']

In [271]:
df = df.drop(columns=['Status Pesanan'])

In [272]:
df['Waktu Pesanan Dibuat'] = pd.to_datetime(df['Waktu Pesanan Dibuat']).dt.date

In [273]:
def extract_variant(product_name):
    match = re.search(r'\b(Ares|Apollo|Athena)\b', product_name, re.IGNORECASE)
    return match.group(0) if match else None

# Terapkan fungsi ke kolom 'Nama Produk'
df['Nama Produk'] = df['Nama Produk'].apply(extract_variant)

In [274]:
# Dictionary referensi koordinat provinsi di Indonesia
provinsi_koordinat = {
    'ACEH': (4.36855, 97.0253),
    'SUMATERA UTARA': (2.1153547, 99.5450974),
    'SUMATERA BARAT': (-0.7399397, 100.8000051),
    'RIAU': (0.2933466, 101.7068294),
    'JAMBI': (-1.6101232, 103.6131203),
    'SUMATERA SELATAN': (-3.0977119, 104.9142684),
    'BENGKULU': (-3.57785, 102.2655),
    'LAMPUNG': (-4.5585849, 105.4068),
    'KEPULAUAN BANGKA BELITUNG': (-2.7410513, 106.4405872),
    'KEPULAUAN RIAU': (3.9456514, 108.1428669),
    'DKI JAKARTA': (-6.2088, 106.8456),
    'JAWA BARAT': (-6.9248, 107.6071),
    'JAWA TENGAH': (-7.150975, 110.3499255),
    'DI YOGYAKARTA': (-7.797068, 110.3705293),
    'JAWA TIMUR': (-7.24917, 112.75083),
    'BANTEN': (-6.1202, 106.1505),
    'BALI': (-8.3405, 115.092),
    'NUSA TENGGARA BARAT': (-8.65293340, 117.36164760),
    'NUSA TENGGARA TIMUR': (-9.5018, 119.8256),
    'KALIMANTAN BARAT': (-0.0263, 109.3333),
    'KALIMANTAN TENGAH': (-1.6814878, 113.3823545),
    'KALIMANTAN SELATAN': (-3.092641, 114.603416),
    'KALIMANTAN TIMUR': (0.5247839, 116.9317887),
    'KALIMANTAN UTARA': (3.0166, 116.3319),
    'SULAWESI UTARA': (1.4538, 124.8917),
    'SULAWESI TENGAH': (-0.8375, 121.6255),
    'SULAWESI SELATAN': (-5.1476651, 119.4221),
    'SULAWESI TENGGARA': (-4.1434, 122.1746),
    'GORONTALO': (0.6371, 123.262),
    'SULAWESI BARAT': (-2.9521, 119.3793),
    'MALUKU': (-3.2385, 129.475),
    'MALUKU UTARA': (1.570999, 127.5101),
    'PAPUA': (-4.269928, 133.550003),
    'PAPUA BARAT': (-3.3333, 132.75),
    'PAPUA TENGAH': (-3.75, 137.0),
    'PAPUA PEGUNUNGAN': (-4.75, 140.0),
    'PAPUA SELATAN': (-6.75, 134.0),
    'PAPUA BARAT DAYA': (-0.875, 131.25),
    'NUSA TENGGARA BARAT (NTB)': (-8.652933, 116.5604),
    'BANGKA BELITUNG': (-2.7410513, 106.4405872),
    'KEPULAUAN BANGKA BELITUNG (BABEL)': (-2.7410513, 106.4405872)
}

# Tambahkan kolom Latitude dan Longitude berdasarkan Provinsi
df['Latitude'] = df['Provinsi'].map(lambda x: provinsi_koordinat.get(x, (None, None))[0])
df['Longitude'] = df['Provinsi'].map(lambda x: provinsi_koordinat.get(x, (None, None))[1])

In [275]:
df['Total Harga Produk'].info()

<class 'pandas.core.series.Series'>
Index: 219 entries, 241225P5HMVBDD to 2502279HXFM7SK
Series name: Total Harga Produk
Non-Null Count  Dtype 
--------------  ----- 
219 non-null    object
dtypes: object(1)
memory usage: 3.4+ KB


In [276]:
df['Marketplace'] = 'Shopee'
df['Total Harga Produk'] = df['Total Harga Produk'].str.replace('Rp', '').str.replace('.', '')
df['Total Harga Produk'] = df['Total Harga Produk'].astype(float)
# df['Total Harga Produk'] = df.apply(
#     lambda row: row['Harga Setelah Diskon'] * row['Jumlah']
    
#     if pd.isnull(row['Total Harga Produk']) 
#     else row ['Total Harga Produk'], axis=1)

df['Total Harga Produk'] = df['Harga Setelah Diskon'] * df['Jumlah'] * 1000

In [277]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 219 entries, 241225P5HMVBDD to 2502279HXFM7SK
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Waktu Pesanan Dibuat        219 non-null    object 
 1   Nama Produk                 219 non-null    object 
 2   Harga Awal                  219 non-null    float64
 3   Harga Setelah Diskon        219 non-null    float64
 4   Jumlah                      219 non-null    int64  
 5   Total Harga Produk          219 non-null    float64
 6   Diskon Dari Penjual         219 non-null    float64
 7   Diskon Dari Shopee          219 non-null    int64  
 8   Voucher Ditanggung Penjual  219 non-null    int64  
 9   Voucher Ditanggung Shopee   219 non-null    float64
 10  Username (Pembeli)          219 non-null    object 
 11  Kota/Kabupaten              219 non-null    object 
 12  Provinsi                    219 non-null    object 
 13  Latitude        

In [278]:
df['Jumlah'] = df['Jumlah'].astype(int)

columns_to_multiply = [
    'Harga Awal',
    'Harga Setelah Diskon',
    'Diskon Dari Penjual',
    'Diskon Dari Shopee',
    'Voucher Ditanggung Penjual',
    'Voucher Ditanggung Shopee'
]

# Mengalikan setiap kolom dalam daftar dengan 1000
df[columns_to_multiply] = df[columns_to_multiply].apply(lambda x: x * 1000)


In [279]:
df2 = df.copy()

In [280]:
start_date = df2['Waktu Pesanan Dibuat'].min()
end_date = df2['Waktu Pesanan Dibuat'].max()
all_dates = pd.date_range(start=start_date, end=end_date, freq='D')

In [281]:
# Menentukan rentang tanggal dari dataset
date_range = pd.date_range(start=df["Waktu Pesanan Dibuat"].min(), 
                           end=df["Waktu Pesanan Dibuat"].max(), 
                           freq='D')

# Membuat DataFrame baru dengan rentang tanggal lengkap
full_dates_df = pd.DataFrame({"Waktu Pesanan Dibuat": date_range})

# Konversi kolom 'Waktu Pesanan Dibuat' di df menjadi datetime
df["Waktu Pesanan Dibuat"] = pd.to_datetime(df["Waktu Pesanan Dibuat"])

# Menggabungkan dengan dataset utama untuk memastikan setiap tanggal ada dalam data
df_full = full_dates_df.merge(df, on="Waktu Pesanan Dibuat", how="left")

# Mengisi nilai NaN dengan 0 untuk kolom jumlah produk yang terjual
df_full["Jumlah"] = df_full["Jumlah"].fillna(0)
df_full["Total Harga Produk"] = df_full["Total Harga Produk"].fillna(0)

In [282]:
df_full['Waktu Pesanan Dibuat'] = pd.to_datetime(df_full['Waktu Pesanan Dibuat']).dt.date

In [283]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Waktu Pesanan Dibuat        227 non-null    object 
 1   Nama Produk                 219 non-null    object 
 2   Harga Awal                  219 non-null    float64
 3   Harga Setelah Diskon        219 non-null    float64
 4   Jumlah                      227 non-null    float64
 5   Total Harga Produk          227 non-null    float64
 6   Diskon Dari Penjual         219 non-null    float64
 7   Diskon Dari Shopee          219 non-null    float64
 8   Voucher Ditanggung Penjual  219 non-null    float64
 9   Voucher Ditanggung Shopee   219 non-null    float64
 10  Username (Pembeli)          219 non-null    object 
 11  Kota/Kabupaten              219 non-null    object 
 12  Provinsi                    219 non-null    object 
 13  Latitude                    219 non

In [284]:
# Daftar varian parfum
varian_parfum = ["Ares", "Athena", "Apollo"]

# Membuat DataFrame dengan kombinasi semua tanggal dan varian parfum
full_dates_products = pd.MultiIndex.from_product(
    [df_full["Waktu Pesanan Dibuat"].unique(), varian_parfum], 
    names=["Waktu Pesanan Dibuat", "Nama Produk"]
).to_frame(index=False)

# Menggabungkan dengan dataset utama untuk memastikan setiap tanggal memiliki semua varian parfum
df_complete = full_dates_products.merge(df_full, on=["Waktu Pesanan Dibuat", "Nama Produk"], how="left")

# Mengisi nilai NaN dengan 0 untuk kolom jumlah produk yang terjual
fill_columns = ["Jumlah", "Total Harga Produk", "Diskon Dari Penjual", "Diskon Dari Shopee"]
df_complete[fill_columns] = df_complete[fill_columns].fillna(0)

In [285]:
# Mengisi nilai NaN dengan dummy values yang diberikan
dummy_values = {
    'Harga Awal': 0.0,
    'Harga Setelah Diskon': 0.0,
    'Jumlah': 0,
    'Total Harga Produk': 0.0,
    'Diskon Dari Penjual': 0.0,
    'Diskon Dari Shopee': 0.0,
    'Voucher Ditanggung Penjual': 0.0,
    'Voucher Ditanggung Shopee': 0.0,
    'Username (Pembeli)': 'dummy_user',
    'Kota/Kabupaten': 'dummy_city',
    'Provinsi': 'dummy_province',
    'Latitude': 0.0,
    'Longitude': 0.0,
    'Marketplace': 'Shopee'
}

df_complete.fillna(dummy_values, inplace=True)


In [286]:
df_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295 entries, 0 to 294
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Waktu Pesanan Dibuat        295 non-null    object 
 1   Nama Produk                 295 non-null    object 
 2   Harga Awal                  295 non-null    float64
 3   Harga Setelah Diskon        295 non-null    float64
 4   Jumlah                      295 non-null    float64
 5   Total Harga Produk          295 non-null    float64
 6   Diskon Dari Penjual         295 non-null    float64
 7   Diskon Dari Shopee          295 non-null    float64
 8   Voucher Ditanggung Penjual  295 non-null    float64
 9   Voucher Ditanggung Shopee   295 non-null    float64
 10  Username (Pembeli)          295 non-null    object 
 11  Kota/Kabupaten              295 non-null    object 
 12  Provinsi                    295 non-null    object 
 13  Latitude                    295 non

In [287]:
df_complete.columns = ['Tanggal Order', 'Nama Produk', 'Harga Awal', 'Harga Setelah Diskon', 'Jumlah', 'Omzet', 'Diskon dari Seller', 'Diskon dari Platform', 'Voucher dari Seller', 'Voucher dari Platform', 'Username Buyer', 'Kota/Kabupaten', 'Provinsi', 'Latitude', 'Longitude', 'Channel']  

In [288]:
df_complete.head()

,Tanggal Order,Nama Produk,Harga Awal,Harga Setelah Diskon,Jumlah,Omzet,Diskon dari Seller,Diskon dari Platform,Voucher dari Seller,Voucher dari Platform,Username Buyer,Kota/Kabupaten,Provinsi,Latitude,Longitude,Channel
0,2024-12-25,Ares,198000.0,188000.0,1.0,188000.0,10000.0,0.0,0.0,84600.0,divemantis,KOTA TANGERANG,BANTEN,-6.120200,106.150500,Shopee
1,2024-12-25,Ares,198000.0,188000.0,1.0,188000.0,10000.0,0.0,0.0,28200.0,sendyaliyanto,KAB. BOGOR,JAWA BARAT,-6.924800,107.607100,Shopee
2,2024-12-25,Ares,198000.0,188000.0,1.0,188000.0,10000.0,0.0,0.0,28200.0,mirav123,KAB. SLEMAN,DI YOGYAKARTA,-7.797068,110.370529,Shopee
3,2024-12-25,Athena,198000.0,188000.0,1.0,188000.0,10000.0,0.0,0.0,84600.0,divemantis,KOTA TANGERANG,BANTEN,-6.120200,106.150500,Shopee
4,2024-12-25,Athena,198000.0,188000.0,1.0,188000.0,10000.0,0.0,0.0,28200.0,aldibrozol,KOTA JAKARTA UTARA,DKI JAKARTA,-6.208800,106.845600,Shopee


In [289]:
df['Harga Awal'] = df['Harga Awal'] * 1000

In [290]:
df['Harga Awal'].head()

No. Pesanan
241225P5HMVBDD    198000000.0
241225P5HMVBDD    198000000.0
241225P5HMVBDD    198000000.0
241225P5WAQ6A5    198000000.0
241225PAMJKABU    198000000.0
Name: Harga Awal, dtype: float64

In [291]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 219 entries, 241225P5HMVBDD to 2502279HXFM7SK
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Waktu Pesanan Dibuat        219 non-null    datetime64[ns]
 1   Nama Produk                 219 non-null    object        
 2   Harga Awal                  219 non-null    float64       
 3   Harga Setelah Diskon        219 non-null    float64       
 4   Jumlah                      219 non-null    int32         
 5   Total Harga Produk          219 non-null    float64       
 6   Diskon Dari Penjual         219 non-null    float64       
 7   Diskon Dari Shopee          219 non-null    int64         
 8   Voucher Ditanggung Penjual  219 non-null    int64         
 9   Voucher Ditanggung Shopee   219 non-null    float64       
 10  Username (Pembeli)          219 non-null    object        
 11  Kota/Kabupaten              219 non-nul

In [292]:
df_complete.to_excel('shopee_cleaned.xlsx', index=False)

In [293]:
# df_complete.T.to_excel('shopee_transposed.xlsx')

In [294]:
df_aggregated = df_complete.copy()

df_complete["Tanggal Order"] = pd.to_datetime(df_complete["Tanggal Order"]).dt.date

df_aggregated = df_complete.groupby(["Tanggal Order", "Nama Produk"])["Jumlah"].sum().reset_index()

date_product_pivot = df_aggregated.pivot(index="Tanggal Order", columns="Nama Produk", values="Jumlah").fillna(0)

date_product_pivot = date_product_pivot.reset_index()

date_product_pivot['Tanggal Order'] = pd.to_datetime(date_product_pivot['Tanggal Order']).dt.date # Ambil tanggal saja
# date_product_pivot['Marketplace'] = 'Shopee'

# Convert the pivoted DataFrame back to long format with only two columns
df_long_format = date_product_pivot.melt(id_vars=["Tanggal Order"], var_name="Nama Produk", value_name="Jumlah")
df_long_format['Marketplace'] = 'Shopee'
# Sort the DataFrame by date in ascending order
df_long_format = df_long_format.sort_values(by="Tanggal Order")

# export_path = "missing_date_filled.xlsx"
# df_fixed.to_excel(export_path, index=False)
# df_long_format.to_excel("long_format_data.xlsx", index=False)  

In [295]:
date_product_pivot['Marketplace'] = 'Shopee'
date_product_pivot.to_excel("shopee_rekap.xlsx", index=False)
